## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-19-22-45-48 +0000,nyt,"Once a Source of Life and Renewal, Monsoon Bri...",https://www.nytimes.com/2025/08/19/world/asia/...
1,2025-08-19-22-44-46 +0000,nypost,Single mom who is a quadruplet gives birth to ...,https://nypost.com/2025/08/19/us-news/single-m...
2,2025-08-19-22-40-41 +0000,startribune,Minneapolis City Council member makes expletiv...,https://www.startribune.com/mpls-city-council-...
3,2025-08-19-22-36-17 +0000,startribune,U-led vaccine project seeks to counter federal...,https://www.startribune.com/u-led-vaccine-proj...
4,2025-08-19-22-35-28 +0000,nypost,Enraged pal of arrested RI prosecutor was tryi...,https://nypost.com/2025/08/19/us-news/enraged-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2371/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
27,trump,53
159,ukraine,31
285,zelensky,17
95,house,16
207,deal,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
272,2025-08-19-02-43-00 +0000,wsj,Trump Pushes for Putin and Zelensky Meeting in...,https://www.wsj.com/world/trump-pushes-for-pea...,141
193,2025-08-19-14-07-00 +0000,wsj,"After months of effort, President Trump says a...",https://www.wsj.com/world/trump-plans-to-get-p...,131
73,2025-08-19-20-30-23 +0000,latimes,Contributor: Trump's Russia and Ukraine summit...,https://www.latimes.com/opinion/story/2025-08-...,124
81,2025-08-19-20-23-41 +0000,bbc,Kremlin plays down Zelensky talks as Trump war...,https://www.bbc.com/news/articles/cn92e52rpjxo...,118
221,2025-08-19-10-25-29 +0000,nypost,Putin and Zelensky could meet within next two ...,https://nypost.com/2025/08/19/world-news/putin...,118


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
272,141,2025-08-19-02-43-00 +0000,wsj,Trump Pushes for Putin and Zelensky Meeting in...,https://www.wsj.com/world/trump-pushes-for-pea...
21,76,2025-08-19-21-56-00 +0000,wsj,"Flight Attendants Win Boarding Pay, Four-Year ...",https://www.wsj.com/business/airlines/air-cana...
267,64,2025-08-19-04-24-10 +0000,nypost,White House mocks ‘unserious’ NJ Dem Rep. LaMo...,https://nypost.com/2025/08/19/us-news/white-ho...
154,50,2025-08-19-16-34-37 +0000,nypost,First biography from Oct. 7 hostage Eli Sharab...,https://nypost.com/2025/08/19/world-news/first...
178,49,2025-08-19-14-54-58 +0000,nyt,Netanyahu Faces Pressure From Far Right Over N...,https://www.nytimes.com/2025/08/19/world/middl...
50,36,2025-08-19-21-04-20 +0000,nypost,Pro-Adams PAC calling NYC voters to rip ‘sore ...,https://nypost.com/2025/08/19/us-news/pro-adam...
11,36,2025-08-19-22-22-53 +0000,nypost,Married Ohio teachers killed in fiery out-of-s...,https://nypost.com/2025/08/19/us-news/married-...
294,33,2025-08-19-00-01-00 +0000,wsj,The Trump administration has restored a websit...,https://www.wsj.com/politics/policy/lawmakers-...
234,32,2025-08-19-09-39-52 +0000,bbc,"Watch: Two presidents, two very different Oval...",https://www.bbc.com/news/videos/c74173p9nexo?a...
198,32,2025-08-19-13-43-36 +0000,nyt,$90 billion in weapons would boost Ukraine aga...,https://www.nytimes.com/live/2025/08/19/us/tru...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
